In [ ]:
#DATA PREPROCESSING
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load data
df = pd.read_csv('route_data.csv')

# Define features
numerical = ['fuel_usage', 'cargo_weight', 'temperature', 'humidity', 'time_taken']
categorical = ['vehicle_type', 'traffic_condition']

# Pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical),
    ('cat', cat_pipeline, categorical)
])

X = df.drop(columns='carbon_emission')
y = df['carbon_emission']


In [ ]:
#MODEL TRAINING(DEEP LEARNING)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Preprocess
X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2)

# Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Regression output
])
model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])

# Train
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2)


In [ ]:
#EVALUATION METRICS
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
rmse = np.sqrt(mean_squared_error(y_test, preds))
percent_error = np.mean(np.abs((y_test - preds.flatten()) / y_test)) * 100

print(f"MAE: {mae:.2f} kg CO₂")
print(f"RMSE: {rmse:.2f} kg CO₂")
print(f"Average % Error: {percent_error:.2f}%")

In [ ]:
#Route Suggestion Engine (Pseudo-Workflow)
def suggest_route(routes_df):
    # Preprocess routes
    processed = preprocessor.transform(routes_df)
    
    # Predict emissions
    predicted_emissions = model.predict(processed)
    routes_df['predicted_emissions'] = predicted_emissions

    # Suggest the lowest-emission route
    best_route = routes_df.loc[routes_df['predicted_emissions'].idxmin()]
    return best_route
